<h1>MLTS Exercise 05 - Unscented Kalman Filter</h1>

This is an optional part of the 5th exercise, where the task is to implement an Unscented Kalman Filter.

## Welcome to Pixel-World

This is Bob.  

<img src="./utils/Bob.png" alt="Map" width="100"/>

Bob is on an adventure and traveling through Pixel-World to find and collect small monsters. As he does not want to get lost on his way, he takes his GPS device with him. Unfortunately, the device can sometimes be unreliable and shows him the wrong position. Therefore he needs an object tracking application which can predict the real path he is taking.  

Your goal is to implement this application and help Bob traverse the following area:  
 
<img src="./utils/Map.png" alt="Map" width="500"/>

The images of this exercie were generated with: [Pokemon Styl Map Generator](https://screensmith.itch.io/pokemon-style-map-generator)

First, lets start with some imports. 

In [ ]:
%load_ext autoreload
%autoreload 2

# import packages
import numpy as np
import matplotlib.pyplot as plt
from filterpy import kalman

In [ ]:
# import helper functions
from utils.helpers import setup_animations
from utils.functions import create_movement

To have an interactive display of your results, run this cell.

In [ ]:
# set display parameters
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150
plt.rcParams['animation.embed_limit'] = 200
plt.ioff()
pass

## Task: Implement an Unscented Kalman Filter

To implement the Unscented Kalman Filter, we will be utilizing a package called [`FilterPy`](https://filterpy.readthedocs.io/en/latest/).  

In FilterPy, you have to first set the initial state and covariance, then the process and measurement noise as well as the state and measurement transition matrices. Then you need to do the recursive steps on the given observations as seen in the lecture or exercise slides. Utilize the predefined function below.

In [ ]:
def state_transition(state: np.ndarray, dt: int, turn_command: int) -> np.ndarray:
    """State transition function.

    Args:
        state (np.ndarray): Previous State z_n-1
        dt (int): Sampling step
        turn_command (int): Turn command (0, 1, -1)

    Returns:
        np.ndarray: Next State z_n
    """
    x, y, vx, vy = state

    # Apply the turn command
    if turn_command == 1:  # Turn right (90° clockwise)
        vx, vy = vy, -vx
    elif turn_command == -1:  # Turn left (90° counterclockwise)
        vx, vy = -vy, vx

    # Update position based on (possibly modified) velocity
    x += vx * dt
    y += vy * dt

    return np.array([x, y, vx, vy])

In [ ]:
def unscented_kalman_filter(zs: np.ndarray, turn_command: np.ndarray, dt: int, Q: np.ndarray, R: np.ndarray, initMu: np.ndarray, initCov: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """Unscented Kalman Filter for a simplified process.

    Args:
        zs (np.ndarray): observations rowise
        turn_command (np.ndarray): contains the turn command by Bob
        dt (int): Delta time.
        Q (np.ndarray): process noise covariance matrix
        R (np.ndarray): measurement noise covariance matrix
        initMu (np.ndarray): initial estimate of a posteriori expectation
        initCov (np.ndarray): initial estimate of a posteriori covariance matrix

    Returns:
        (np.ndarray,np.ndarray): posteriori predictions, posteriori error covs
    """

    # get the correct shapes
    numObservations = zs.shape[0]
    dim_state = Q.shape[0]  # shape of state (position x and y, velocity x and y)
    dim_obs = zs.shape[1]  # shape of observations

    # Define arrays to store posteriors
    posterioriPred = np.empty((numObservations, dim_state))
    posterioriErrorCov = np.empty((numObservations, dim_state, dim_state))

    # Create sigma points for UKF
    points = kalman.MerweScaledSigmaPoints(n=dim_state, alpha=0.1, beta=2.0, kappa=0)

    # Initialize UKF
    ukf = kalman.UnscentedKalmanFilter(dim_x=dim_state, dim_z=dim_obs, dt=dt,
                                       fx=state_transition,
                                       hx=lambda x: x[:2], points=points)

    # Initial state estimate and covariance
    ukf.x = initMu  # Initial state (position, velocity)
    ukf.P = initCov  # Initial covariance (uncertainty in the state)

    # Set matrices
    ukf.Q = Q  # Process noise covariance
    ukf.R = R  # Measurement noise covariance

    # go through observations
    for k in range(numObservations):

        # Predict the next state based on the state transition matrix
        ukf.predict(fx=state_transition, turn_command=turn_command[k])

        # Update the state estimate with the new observation
        ukf.update(zs[k])

        # store the posteriors
        posterioriPred[k] = ukf.x
        posterioriErrorCov[k] = ukf.P

    return posterioriPred, posterioriErrorCov

## Test your object tracking application

To test if your implemented object tracking application (i.e. Kalman Filter), we let Bob walk through the shown area, starting from various postions and taking different paths. We can also adjust the measurements of the GPS device with different levels of noise.

Let's start with some general fixed parameters:

In [ ]:
# set parameters
config = {
    'numMeasurements': 21,  # number of measurements over the distance
    'distanceStartEnd': 21,  # distance tracking object travels
    'msPerTimestep': 500,  # Number of ms per timestep
    'seed': 1234  # Random seed
}

These parameters define the ground truth movement and the observations the model will see.  
You should play around with those values and test out different combinations.

In [ ]:
config = {
    **config,
    'model': 'movement_3',  # Movement model. Can be [movement_1, movement_2, movement_3]
    'varObservations': 0.5,  # Variance in x and y direction. Possible values [0.01, 0.001, 0.0001]
    'processNoiseScaling': 0.0  # Scaling for process noise model in filter. Possible values [0.01, 0.001, 0.0001]
}

Next, here are some parameters that influence your results and your model. Choose these values wisely:

In [ ]:
config = {
    **config,
    'observationNoiseScaling': 5,  # Scaling for observation noise model in filter.
    'initMu': np.array([9, 4, 1, 1]),  # Initial value for state (prior)
    'initCov': np.eye(4) * 1,  # Initial value for covariance (prior)
}

Now, we set up the function for plotting Bobs movement. `data_for_plotting` take the configs defined above, generates the movement of Bob and then runs the object tracking algorithm you implemented.  

<b>Be carefull that you correctly set up and pass the right parameters for the object tracking functions!</b>

In [ ]:
def data_for_plotting(parameters: dict) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Generate the data, predict the movement and plot it.

    Args:
        parameters (dict): Parameters.

    Returns:
        tuple[np.ndarray, np.ndarray, np.ndarray]: Result of the filter algorithm, ground truth movement, observations of the movement.
    """
    # set up everything
    np.random.seed(parameters['seed'])
    samplingPeriod = parameters['distanceStartEnd'] / parameters['numMeasurements']

    # sample some observations and ground truth movement
    observation, referenceCurve = create_movement(parameters['model'], parameters['numMeasurements'],
                                                  samplingPeriod, parameters['varObservations'], parameters['processNoiseScaling'])

    # initialize the measurement and process noise matrices
    Q = parameters['processNoiseScaling'] * np.eye(4)
    R = parameters['observationNoiseScaling'] * np.eye(2)

    # create turn commands by Bob
    turn_command = np.zeros(observation.shape[0])
    turn_command[11] = 1

    # this is the uncented kalman filter
    res = unscented_kalman_filter(observation, turn_command, samplingPeriod, Q, R, parameters['initMu'], parameters['initCov'])

    return res, referenceCurve, observation

### Run Bob, Run!

We will now animate the movement of Bob. The following graph shows his x and y positions over time. The black line is his true movement while the green dots are the observations by the GPS device.  
In red you see the predictions of your tracking algorithm with the uncertainty as a circle around it. Use the buttons below to start and stop the animation.

In [ ]:
aniFiltered = setup_animations(data_for_plotting(config), config)
plt.close('all')
aniFiltered

### Questions:

One of the movement cannot be tracked by the Kalman Filter. Which other object tracking algorithm could you use there?  
How does our transition and mesurement model change?  
Can you implement it?  
What information do you need to add to get it working?